
From the book (chapter 14.1):

> In 2011, URL shortening service [Bitly](https://bitly.com/) partnered with the US government website [USA.gov](https://www.usa.gov/) to provide a feed of anonymous data gathered from users who shorten links ending with .gov or .mil. In 2011, a live feed as well as hourly snapshots were available as downloadable text files. This service is shut down at the time of this writing (2017)

___



In [ ]:
library(tidyverse)
library(magrittr)
library(jsonlite)


___

## Loading the data

The dataset is in [JSON](https://www.json.org) format, so it is imported using the `jsonlite` package. Its `fromJSON` function however does not work in this case, since this is a NDJSON file, meaning that multiple JSON objects are contained in the same file.
We will use the `stream_in` function to load the dataset instead.



In [ ]:
path <- "../datasets/bitly_usagov/example.txt"
records <- stream_in(file(path))


Let's convert the `records` object to a tibble.



In [ ]:
records %<>% as_tibble()

In [ ]:
records



## Counting time zones

We may be interested in counting the most often-occurring time zones.



In [ ]:
tz_counts <- records %>%
    count(tz)

In [ ]:
tz_counts



There are some missing values, so let's first replace them.



In [ ]:
records %<>%
    mutate(tz = replace(tz, tz == "", "Missing"))
records$tz %<>%
    replace_na("Unknown")

In [ ]:
tz_counts <- records %>%
    count(tz)

In [ ]:
tz_counts


Now we can sort entries in descending order.



In [ ]:
tz_counts %<>%
    arrange(desc(n))


Let's plot a bar chart with the first 10 entries from `tz_counts`.



In [ ]:
head(tz_counts, n = 10) %>%
    ggplot(aes(x = reorder(tz, n), y = n, fill = tz)) +
    geom_col() +
    coord_flip() +
    guides(fill = F) +
    labs(title = "Top time zones", x = "Time zone", y = "Count")


Let's parse the `a` column to get some information about the most commonly used browsers.



In [ ]:
records %<>%
    mutate(browsers = str_split_fixed(records$a, " ", n = 2)[, 1])

In [ ]:
records %>%
    count(browsers) %>%
    arrange(desc(n))


Now we want to decompose time zones into Windows and non-Windows users. We can achieve that by looking at the `a` variable, checking whether it contains the `Windows` string or not. Some of the browser agents are missing, so we'll first drop rows where the `a` column is empty.



In [ ]:
records_notnull <- records %>%
    drop_na(a)


Then we'll create a new column, `os`, which will contain the `Win` value for Windows users and `NonWin` otherwise.



In [ ]:
records_notnull %<>%
    mutate(os = case_when(str_detect(a, "Windows") ~ "Win",
                          TRUE ~ "NonWin"))


Now we'll aggregate these results to show the count of Windows and non-Windows users for each time zone.



In [ ]:
agg_counts <- records_notnull %>%
    group_by(tz, os) %>%
    summarise(Users = n()) %>%
    spread(os, Users, fill = 0)

In [ ]:
agg_counts


Let's add a `total` column that will be useful to sort our data based on the total number of users; for visualization purposes, we'll take the first 10 entries in this list.



In [ ]:
agg_counts %<>%
    mutate(total = sum(NonWin, Win)) %>%
    arrange(desc(total))

agg_counts_top10 <- agg_counts[1:10, ]


In order to visualize these data, let's gather again `Win` and `NonWin` counts per each time zone.



In [ ]:
count_subset <- agg_counts_top10 %>%
    gather(key = "os", value = "users", Win, NonWin) %>%
    arrange(tz, users) %>%
    select(-total)


Now we can plot these data.



In [ ]:
count_subset %>%
    ggplot(aes(x = reorder(tz, users), y = users, fill = os)) +
    geom_col(position = "dodge") +
    coord_flip() +
    labs(x = "Time zone", y = "Users",
         title = "Windows and non-Windows users per time zone", fill = "OS") +
    theme(legend.position = c(0.9, 0.14),
          legend.background = element_rect(fill = "transparent"))


Let's normalize the count of users so the sum of Windows and non-Windows users is 1.



In [ ]:
count_subset %<>%
    group_by(tz, os) %>%
    summarise(users = users) %>%
    mutate(normed_total = users / sum(users))


Now we can plot these normalized data.



In [ ]:
count_subset %>%
    ggplot(aes(x = reorder(tz, users), y = normed_total, fill = os)) +
    geom_col(position = "dodge") +
    coord_flip() +
    labs(x = "Time zone", y = "User ratio",
         title = "Windows and non-Windows users ratio per time zone", fill = "OS") +
    theme(legend.position = c(0.9, 0.86),
          legend.background = element_rect(fill = "transparent"))


___



In [ ]:
sessionInfo()